In [1]:
import re, sys, os
import logging
from pathlib import Path
from datetime import datetime
import torch
import torch_directml
from fastai.collab import *
from fastai.metrics import *
from fastai.data.external import *

torch._logging.set_logs(all=logging.WARNING)

In [2]:
print("="*80)
print("DIRECTML FULL DEBUG LOG STARTED")
print(f"torch version: {torch_directml.torch.__version__}")
print(f"Device: {torch_directml.device()}")
print(f"GPU: {torch_directml.device_name(0)}")
print("="*80)

DIRECTML FULL DEBUG LOG STARTED
torch version: 2.4.1+cpu
Device: privateuseone:0
GPU: Radeon RX 560X 


In [3]:
# 1. Detect DirectML device (no global default!)
dml = torch_directml.device()

In [4]:
path = untar_data(URLs.ML_SAMPLE)

In [5]:
dls = CollabDataLoaders.from_csv(
    path / 'ratings.csv', bs=64, device=dml,
    verbose=True, num_workers=0
)

Setting up after_item: Pipeline: 
Setting up before_batch: Pipeline: 
Setting up after_batch: Pipeline: ReadTabBatch


In [6]:
# For example, accuracy within a threshold
def rating_accuracy(inp, targ):
    """Percentage of predictions within 0.5 of actual rating"""
    return ((inp - targ).abs() < 0.5).float().mean()

In [7]:
learn = collab_learner(
    dls, y_range=(0.5,5.5), metrics=[rmse]
).to_fp16(enabled=False)
learn.to(dml)
""

''

In [8]:
# learn.fit_one_cycle instead of learn.fine_tune
# because we do not have a pretrained model for tabular data
learn.fit_one_cycle(20, lr_max=5e-3)

f:\Projects\VsCodium\Python\2025\GenAI\11\.venv3_12_9\Lib\site-packages\torch\amp\autocast_mode.py:265: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
f:\Projects\VsCodium\Python\2025\GenAI\11\.venv3_12_9\Lib\site-packages\torch\amp\grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


epoch,train_loss,valid_loss,_rmse,time
0,1.504695,1.585330,1.259099,00:02
1,1.393090,1.324470,1.150856,00:02
2,0.933960,0.727966,0.853209,00:02
3,0.644093,0.687102,0.828916,00:02
4,0.520685,0.686696,0.828671,00:02
5,0.407841,0.703819,0.838939,00:02
6,0.322116,0.720423,0.848778,00:02
7,0.263522,0.738108,0.859132,00:02
8,0.207502,0.750632,0.866390,00:02
9,0.178222,0.755140,0.868988,00:02


In [9]:
learn.show_results()

f:\Projects\VsCodium\Python\2025\GenAI\11\.venv3_12_9\Lib\site-packages\torch\amp\autocast_mode.py:265: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
f:\Projects\VsCodium\Python\2025\GenAI\11\.venv3_12_9\Lib\site-packages\torch\amp\grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


,userId,movieId,rating,rating_pred
0,80.0,40.0,3.5,3.691799
1,63.0,82.0,4.0,3.596038
2,9.0,62.0,3.5,4.030405
3,62.0,35.0,4.0,3.539839
4,96.0,51.0,4.0,3.725155
5,98.0,87.0,4.0,3.551827
6,94.0,27.0,4.0,3.603968
7,38.0,83.0,5.0,3.921100
8,36.0,82.0,5.0,4.555249
